In [4]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import tensorflow as tf
print("TensorFlow version: ",tf.version.VERSION)

import pandas as pd
import numpy as np
import urllib
import json
import datetime

from utils import get_daily_demand_data
import os

TensorFlow version:  2.3.4


In [2]:
!pip install --user google-cloud-bigquery==1.25.0

     |████████████████████████████████| 169 kB 5.8 MB/s eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-storage 1.42.0 requires google-resumable-media<3.0dev,>=1.3.0; python_version >= "3.6", but you have google-resumable-media 0.5.1 which is incompatible.


In [5]:
# Change with your own bucket and project below:
BUCKET =  "qwiklabs-gcp-02-68978b5afbca"
PROJECT = "qwiklabs-gcp-02-68978b5afbca"

OUTDIR = "gs://{bucket}/demand/data".format(bucket=BUCKET)

os.environ['BUCKET'] = BUCKET
os.environ['OUTDIR'] = OUTDIR
os.environ['PROJECT'] = PROJECT

In [6]:
start_date = '2019-04-01'
end_date = '2020-03-31' 

NORM_VALUE = 1e6 # divided all values normalization

values, times = get_daily_demand_data(start_date, end_date, write_json=False)
values = values / NORM_VALUE

In [12]:
dataset = pd.DataFrame({'times': times, 'values': values})
dataset.to_csv("dataset.csv")

In [17]:
!gsutil cp -r data gs://qwiklabs-gcp-02-68978b5afbca

Copying file://data/dataset.csv [Content-Type=text/csv]...
Copying file://data/data.json [Content-Type=application/json]...                
Copying file://data/.ipynb_checkpoints/dataset-checkpoint.csv [Content-Type=text/csv]...
/ [3 files][ 48.8 KiB/ 48.8 KiB]                                                
Operation completed over 3 objects/48.8 KiB.                                     


## Create BQ Dataset

In [14]:
# https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive2/feature_engineering/labs/1_bqml_basic_feat_eng_bqml-lab.ipynb

In [14]:
%%bash
# Create a BigQuery dataset

datasetexists=$(bq ls -d | grep -w demand_pred)

if [ -n "$datasetexists" ]; then
    echo -e "BigQuery dataset already exists, let's not recreate it."

else
    echo "Creating BigQuery dataset"
    
    bq --location=US mk --dataset \
        --description 'Demand prediction' \
        $PROJECT:demand_pred
   echo "Here are your current datasets:"
   bq ls
fi

Creating BigQuery dataset
Dataset 'qwiklabs-gcp-02-68978b5afbca:demand_pred' successfully created.
\nHere are your current datasets:
   datasetId   
 ------------- 
  demand_pred  


In [26]:
%%bash

bq load --autodetect \
        --location=US \
        --source_format=CSV \
        qwiklabs-gcp-02-68978b5afbca:demand_pred.mytable \
        dataset.csv 

Upload complete.
Waiting on bqjob_r4868e967e97a7586_0000017bb0e0dd5a_1 ... (1s) Current status: DONE   


In [20]:
%%bigquery
 
SELECT times, values
FROM
  demand_pred.dataset

Downloading: 100%|██████████| 366/366 [00:01<00:00, 231.92rows/s]


,times,values
0,2019-04-01,0.728854
1,2019-04-02,0.740720
2,2019-04-03,0.743726
3,2019-04-04,0.746532
4,2019-04-05,0.758884
...,...,...
361,2020-03-27,0.667336
362,2020-03-28,0.605617
363,2020-03-29,0.540515
364,2020-03-30,0.610996


## Create Model!

In [ ]:
%%bigquery

CREATE OR REPLACE MODEL
  feat_eng.baseline_model OPTIONS (model_type='linear_reg',
    input_label_cols=['fare_amount']) AS
SELECT
  fare_amount,
  passengers,
  pickup_datetime,
  pickuplon,
  pickuplat,
  dropofflon,
  dropofflat
FROM
  feat_eng.feateng_training_data